# **PROJET ITK-VTK**

## Auteurs
### Majeure IMAGE

- David CHEMALY
    - login: david.chemaly
<br /> <br />
- Thuraya SHANBARI
    - login: thuraya.shanbari

# Partie 2
---

## Import des librairies

In [3]:
import itk
itk.Version.GetITKVersion()

'5.3.0'

In [4]:
import vtk
vtk.vtkVersion.GetVTKVersion()

'9.2.6'

In [5]:
import matplotlib
import matplotlib.pyplot as plt

matplotlib.__version__

'3.7.1'

## Paramètres

In [6]:
image1_path = 'Data/case6_gre1.nrrd'
image2_path = 'Data/case6_gre2.nrrd'

Dimension = 3
PixelType = itk.ctype('float')
ImageType = itk.Image[PixelType, Dimension]

## Chargement des données

In [7]:
def read_file(file_path):
    readerType = itk.ImageFileReader[ImageType]
    reader = readerType.New()
    reader.SetFileName(file_path)
    reader.Update()
    image = reader.GetOutput()
    return image

In [8]:
image1 = read_file(image1_path)
image2 = read_file(image2_path)

print("image1 size: " + str(image1.GetLargestPossibleRegion().GetSize()))
print("image2 size: " + str(image2.GetLargestPossibleRegion().GetSize()))

image1 size: itkSize3 ([256, 256, 176])
image2 size: itkSize3 ([256, 256, 176])


## Affichage des données

In [9]:
def display_image(image, slice_number):
    plt.imshow(image[slice_number,:,:], cmap='gray')
    plt.show()

## Pipeline

### 1. Segmentation Watershed

In [10]:
def segmentation_watershed(image):

    InternalPixelType = itk.F
    InternalImageType = itk.Image[InternalPixelType, Dimension]

    GradientMagnitudeImageFilterType = itk.GradientMagnitudeRecursiveGaussianImageFilter[InternalImageType, InternalImageType]
    gradientMagnitudeImageFilter = GradientMagnitudeImageFilterType.New()
    gradientMagnitudeImageFilter.SetInput(image)
    gradientMagnitudeImageFilter.SetSigma(1.0)

    WatershedFilterType = itk.WatershedImageFilter[InternalImageType]
    watershedFilter = WatershedFilterType.New()

    watershedFilter.SetInput(gradientMagnitudeImageFilter.GetOutput())
    watershedFilter.SetThreshold(0.01)
    watershedFilter.SetLevel(0.2)

    watershedFilter.Update()

    return watershedFilter.GetOutput()

### 2. Segmentation Connected Threshold

In [11]:
def segmentation_connected_threshold(image, seed, lower, upper):

    InternalPixelType = itk.F
    InternalImageType = itk.Image[InternalPixelType, Dimension]

    CurvatureFlowImageFilterType = itk.CurvatureFlowImageFilter[InternalImageType, InternalImageType]
    smoothing = CurvatureFlowImageFilterType.New()

    ConnectedFilterType = itk.ConnectedThresholdImageFilter[InternalImageType, InternalImageType]
    connectedThreshold = ConnectedFilterType.New()

    smoothing.SetInput(image)
    smoothing.SetNumberOfIterations(5)
    smoothing.SetTimeStep(0.125)

    connectedThreshold.SetInput(smoothing.GetOutput())
    connectedThreshold.SetLower(lower)
    connectedThreshold.SetUpper(upper)
    connectedThreshold.SetReplaceValue(255)
    connectedThreshold.AddSeed(seed)

    connectedThreshold.Update()

    return connectedThreshold.GetOutput()